In [ ]:
%mkdir -p /content/drive/MyDrive/GSE241683
%cd /content/drive/MyDrive/GSE241683

/content/drive/MyDrive/GSE241683


# Env setup

In [ ]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
COLS, ORGANISMS

(['cancer_type',
  'crispr_type',
  'cell_type',
  'organism',
  'perturbation_name',
  'condition'],
 ['Humans (Homo sapiens)',
  'Mice (Mus musculus)',
  'Rats (Rattus norvegicus)',
  'Zebrafish (Danio rerio)',
  'Fruit flies (Drosophila melanogaster)',
  'Nematodes (Caenorhabditis elegans)',
  'Yeast (Saccharomyces cerevisiae)',
  'Dogs (Canis lupus familiaris)',
  'Pigs (Sus scrofa)'])

# SubSeries: GSE241838 CAR T cells

In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE241838&format=file" -O GSE241838_RAW.tar

--2024-07-22 13:22:25--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE241838&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241059840 (230M) [application/x-tar]
Saving to: ‘GSE241838_RAW.tar’

GSE241838_RAW.tar   100%[===================>] 229.89M  6.76MB/s    in 24s     

2024-07-22 13:22:50 (9.50 MB/s) - ‘GSE241838_RAW.tar’ saved [241059840/241059840]



In [ ]:
with tarfile.open("GSE241838_RAW.tar") as tar:
    for member in tar.getmembers():
        print(member.name)

GSM7744298_CAR-T_ALL-CELLS.h5ad.gz
GSM7744298_CART_UCSF_aneuploidy_breakpoints.csv.gz
GSM7744298_inferCNV_results_CAR-T_Donor6_Day4.txt.gz
GSM7744298_inferCNV_results_CAR-T_Donor6_Day7.txt.gz
GSM7744298_inferCNV_results_CAR-T_Donor7_Day4.txt.gz
GSM7744298_inferCNV_results_CAR-T_Donor7_Day7.txt.gz
GSM7744298_inferCNVgeneName.txt.gz


In [ ]:
import gzip

with tarfile.open("GSE241838_RAW.tar") as tar:
    gz_file = tar.extractfile("GSM7744298_CAR-T_ALL-CELLS.h5ad.gz")
    with gzip.open(gz_file) as f:
        adata = sc.read_h5ad(f)

adata

AnnData object with n_obs × n_vars = 55213 × 36601
    obs: 'batch', 'barcodes', 'sample_id', 'treatment', 'timepoint', 'donor'
    var: 'gene_ids', 'feature_types'

In [ ]:
adata.obs.head()

,batch,barcodes,sample_id,treatment,timepoint,donor
AAACCCACAGCGAGTA-1,1,AAACCCACAGCGAGTA-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6
AAACCCAGTAGGATAT-1,1,AAACCCAGTAGGATAT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6
AAACCCAGTCCGACGT-1,1,AAACCCAGTCCGACGT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6
AAACGAATCACACGAT-1,1,AAACGAATCACACGAT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6
AAACGAATCTGCGGGT-1,1,AAACGAATCTGCGGGT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6


In [ ]:
adata.obs['treatment'].value_counts()

treatment
exon_minusAAV_donor6_day7      6639
control_gRNA_donor7_day4       5145
exon_minusAAV_donor7_day4      4737
intron_plusAAV_donor6_day4     4318
intron_minusAAV_donor7_day4    4214
exon_minusAAV_donor6_day4      3853
control_gRNA_donor6_day4       3703
intron_minusAAV_donor6_day4    3460
intron_minusAAV_donor7_day7    2727
exon_plusAAV_donor6_day4       2398
intron_plusAAV_donor6_day7     2198
control_gRNA_donor7_day7       2179
control_gRNA_donor6_day7       1832
exon_minusAAV_donor7_day7      1587
exon_plusAAV_donor6_day7       1515
intron_minusAAV_donor6_day7    1500
exon_plusAAV_donor7_day7       1379
exon_plusAAV_donor7_day4       1236
intron_plusAAV_donor7_day4      357
intron_plusAAV_donor7_day7      236
Name: count, dtype: int64

In [ ]:
adata.var_names.str.startswith('sgRNA').any()

False

In [ ]:
adata.obs['organism'] = ORGANISMS[0]
adata.obs['crispr_type'] = 'CRISPR KO'
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = 'CAR T cell'

In [ ]:
def get_perturbation_name(treatment):
    if treatment.startswith('exon') or treatment.startswith('intron'):
        return 'TRAC'
    else:
        return 'non-targeting'

def get_condition(treatment):
    if treatment.startswith('control'):
        return 'control'
    else:
        return "_".join(treatment.split("_")[:2])

In [ ]:
adata.obs['perturbation_name'] = adata.obs['treatment'].apply(get_perturbation_name)
adata.obs['condition'] = adata.obs['treatment'].apply(get_condition)

In [ ]:
adata.write_h5ad("./GSE241683_merged.h5ad")

In [ ]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [ ]:
adata.obs.head()

,batch,barcodes,sample_id,treatment,timepoint,donor,organism,crispr_type,perturbation_name,condition,cancer_type,cell_type
AAACCCACAGCGAGTA-1,1,AAACCCACAGCGAGTA-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACCCAGTAGGATAT-1,1,AAACCCAGTAGGATAT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACCCAGTCCGACGT-1,1,AAACCCAGTCCGACGT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACGAATCACACGAT-1,1,AAACGAATCACACGAT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACGAATCTGCGGGT-1,1,AAACGAATCTGCGGGT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell


In [ ]:
adata.obs.head()

,batch,barcodes,sample_id,treatment,timepoint,donor,organism,crispr_type,perturbation_name,condition,cancer_type,cell_type
AAACCCACAGCGAGTA-1,1,AAACCCACAGCGAGTA-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACCCAGTAGGATAT-1,1,AAACCCAGTAGGATAT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACCCAGTCCGACGT-1,1,AAACCCAGTCCGACGT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACGAATCACACGAT-1,1,AAACGAATCACACGAT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell
AAACGAATCTGCGGGT-1,1,AAACGAATCTGCGGGT-1,well-1_control_gRNA_donor6,control_gRNA_donor6_day4,day4,donor6,Humans (Homo sapiens),CRISPR KO,non-targeting,control,non-cancer,CAR T cell


In [ ]:
perturb_info = pd.read_csv('https://raw.githubusercontent.com/yelabucsf/Cas9_chromosome_loss_scRNA-seq/main/data_processing/feature_reference_v2.csv')
perturb_info

,id,name,read,pattern,sequence,feature_type,target_gene_id,target_gene_name
0,PTPRC-1,PTPRC-1,R2,(BC)GTTTAAGAGCTATG,AGCATTATCCAAAGAGTCCG,CRISPR Guide Capture,ENSG00000081237,PTPRC
1,PTPRC-2,PTPRC-2,R2,(BC)GTTTAAGAGCTATG,TGTGGATTACTTATATAACA,CRISPR Guide Capture,ENSG00000081237,PTPRC
2,PTPRC-3,PTPRC-3,R2,(BC)GTTTAAGAGCTATG,CACAAATACATGGTCATATC,CRISPR Guide Capture,ENSG00000081237,PTPRC
3,PTPRC-4,PTPRC-4,R2,(BC)GTTTAAGAGCTATG,GTATTTGTGGCTTAAACTCT,CRISPR Guide Capture,ENSG00000081237,PTPRC
4,PIK3CD-1,PIK3CD-1,R2,(BC)GTTTAAGAGCTATG,TTTTTAGGACAACTGTCATC,CRISPR Guide Capture,ENSG00000171608,PIK3CD
...,...,...,...,...,...,...,...,...
395,TUBB-4,TUBB-4,R2,(BC)GTTTAAGAGCTATG,GCTGACCACACCAACCTACG,CRISPR Guide Capture,ENSG00000196230,TUBB
396,DHODH-1,DHODH-1,R2,(BC)GTTTAAGAGCTATG,GTGACTCCAAAACCTCAGGA,CRISPR Guide Capture,ENSG00000102967,DHODH
397,DHODH-2,DHODH-2,R2,(BC)GTTTAAGAGCTATG,GAGATGTATGCACTCACCCA,CRISPR Guide Capture,ENSG00000102967,DHODH
398,DHODH-3,DHODH-3,R2,(BC)GTTTAAGAGCTATG,GGGGTTGGTGGTGTGAGCAG,CRISPR Guide Capture,ENSG00000102967,DHODH


# SubSeries: GSE241882 CROP-seq

In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE241882&format=file" -O GSE241882_RAW.tar

--2024-07-23 07:02:07--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE241882&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14638479360 (14G) [application/x-tar]
Saving to: ‘GSE241882_RAW.tar’

GSE241882_RAW.tar   100%[===================>]  13.63G  31.8MB/s    in 8m 22s  

2024-07-23 07:10:29 (27.8 MB/s) - ‘GSE241882_RAW.tar’ saved [14638479360/14638479360]



In [ ]:
!tar -tf GSE241882_RAW.tar

GSM7744878_Batch1-infercnv.observations.txt.gz
GSM7744878_Batch2-infercnv.observations.txt.gz
GSM7744878_Batch3-infercnv.observations.txt.gz
GSM7744878_Batch4-infercnv.observations.txt.gz
GSM7744878_Batch5-infercnv.observations.txt.gz
GSM7744878_Batch6-infercnv.observations.txt.gz
GSM7744878_Batch7-infercnv.observations.txt.gz
GSM7744878_Batch8-infercnv.observations.txt.gz
GSM7744878_Batch9-infercnv.observations.txt.gz
GSM7744878_Concat_InferCNV.pkl.gz
GSM7744878_aneuploidy_events.csv.gz
GSM7744878_fully_processed.h5ad.gz
GSM7744878_inferCNVgeneName.txt.gz
GSM7744878_protospacer_calls_per_cell.csv.gz
GSM7744878_qced.h5ad.gz
GSM7744878_sample1-assignment_confidence_table.csv.gz
GSM7744878_sample2-assignment_confidence_table.csv.gz
GSM7744878_sample3-assignment_confidence_table.csv.gz
GSM7744878_sample4-assignment_confidence_table.csv.gz
GSM7744878_sample5-assignment_confidence_table.csv.gz
GSM7744878_sample6-assignment_confidence_table.csv.gz
GSM7744878_sampleA-filtered_feature_bc_matri

In [ ]:
with tarfile.open("GSE241882_RAW.tar") as tar:
    gz_file = tar.extract("GSM7744878_fully_processed.h5ad.gz")

In [ ]:
!gzip -d "GSM7744878_fully_processed.h5ad.gz"

In [ ]:
adata = sc.read_h5ad("GSM7744878_fully_processed.h5ad")
adata

AnnData object with n_obs × n_vars = 204374 × 37001
    obs: 'batch', 'aggr_barcode', 'num_features', 'feature_call', 'num_umis', 'maxUmi1', 'maxUmi2', 'cell_barcode', 'guidePvalue', 'pvalueAssign', 'guides', 'multi_output', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [ ]:
adata.obs.head()

,batch,aggr_barcode,num_features,feature_call,num_umis,maxUmi1,maxUmi2,cell_barcode,guidePvalue,pvalueAssign,guides,multi_output,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
aggr_barcode,,,,,,,,,,,,,,,,
AAACCCAAGACATCAA-1-0,0,AAACCCAAGACATCAA-1-0,4.0,HTT-2|PRF1-1|GALC-3|Non-target-8,1|2|1|1,2,1,AAACCCAAGACATCAA-1,0.500000,MultipleGuide,Multiple Guides,Blank,1144,2547.0,5.0,0.196309
AAACCCAAGATGACAT-1-0,0,AAACCCAAGATGACAT-1-0,3.0,WFS1-4|PRF1-1|SERPINA1-3,1|1|1,1,1,AAACCCAAGATGACAT-1,0.750000,MultipleGuide,Multiple Guides,CTJD02B,2292,5182.0,874.0,16.866074
AAACCCAAGCGAGTCA-1-0,0,AAACCCAAGCGAGTCA-1-0,4.0,IFNGR1-3|CD164-3|CD70-3|IL2RB-2,1|1|1|1,1,1,AAACCCAAGCGAGTCA-1,0.750000,MultipleGuide,Multiple Guides,CTJD02E,2248,5076.0,842.0,16.587864
AAACCCAAGCGCCTCA-1-0,0,AAACCCAAGCGCCTCA-1-0,32.0,CTLA4-3|CCR5-2|RHOH-1|RHOH-4|LRBA-1|CD164-1|IK...,1|2|1|1|1|1|1|1|3|1|1|1|1|1|1|1|1|1|1|1|2|8|1|...,12,8,AAACCCAAGCGCCTCA-1,0.251722,MultipleGuide,Multiple Guides,CTJD02F,6458,28314.0,1122.0,3.962704
AAACCCAAGTAGCATA-1-0,0,AAACCCAAGTAGCATA-1-0,14.0,IFNGR1-4|TRBC1-4|CD5-1|LAG3-2|TNFRSF1A-3|SERPI...,1|6|1|1|1|2|1|1|1|1|1|1|1|1,6,2,AAACCCAAGTAGCATA-1,0.144531,MultipleGuide,Multiple Guides,CTJD02C,5033,21731.0,972.0,4.472873


In [ ]:
adata.obs['guide'] = adata.obs['feature_call'].apply(lambda x: "+".join(x.split('|')))
adata.obs['guide']

aggr_barcode
AAACCCAAGACATCAA-1-0                     HTT-2+PRF1-1+GALC-3+Non-target-8
AAACCCAAGATGACAT-1-0                             WFS1-4+PRF1-1+SERPINA1-3
AAACCCAAGCGAGTCA-1-0                      IFNGR1-3+CD164-3+CD70-3+IL2RB-2
AAACCCAAGCGCCTCA-1-0    CTLA4-3+CCR5-2+RHOH-1+RHOH-4+LRBA-1+CD164-1+IK...
AAACCCAAGTAGCATA-1-0    IFNGR1-4+TRBC1-4+CD5-1+LAG3-2+TNFRSF1A-3+SERPI...
                                              ...                        
TTTGTTGTCAGGACGA-1-5                        CCR5-1+TGFBR1-4+LAG3-2+PCNT-3
TTTGTTGTCATTGTTC-1-5    FAS-2+SERPINA1-3+USP22-4+TNFRSF11A-4+MALT1-3+C...
TTTGTTGTCCTTCTTC-1-5                     IKBKB-4+B2M-3+WAS-1+Non-target-2
TTTGTTGTCTATCGTT-1-5    TIGIT-2+CCR5-2+VEGFA-4+VPS13B-4+CD4-3+ATP7B-4+...
TTTGTTGTCTTAAGGC-1-5                            BCL10-4+PPP1R12C-3+JAG1-1
Name: guide, Length: 204374, dtype: category
Categories (194325, object): ['ABL1-1', 'ABL1-1+ATP7B-2+FOXP3-2', 'ABL1-1+ATP7B-4+EGR3-3',
                              'ABL1-1

In [ ]:
adata.obs['n_guides'] = adata.obs['num_features'].astype(int)
adata.obs['perturbation_name'] = (
    adata.obs['feature_call']
    .apply(
        lambda x: "+".join([
            xx.split('-')[0] if 'Non-target' not in xx else 'non-targeting'
            for xx in x.split('|')
        ])
    )
)
adata.obs['perturbation_name']

aggr_barcode
AAACCCAAGACATCAA-1-0                          HTT+PRF1+GALC+non-targeting
AAACCCAAGATGACAT-1-0                                   WFS1+PRF1+SERPINA1
AAACCCAAGCGAGTCA-1-0                              IFNGR1+CD164+CD70+IL2RB
AAACCCAAGCGCCTCA-1-0    CTLA4+CCR5+RHOH+RHOH+LRBA+CD164+IKBKB+VPS13B+V...
AAACCCAAGTAGCATA-1-0    IFNGR1+TRBC1+CD5+LAG3+TNFRSF1A+SERPINA1+CCR7+M...
                                              ...                        
TTTGTTGTCAGGACGA-1-5                                CCR5+TGFBR1+LAG3+PCNT
TTTGTTGTCATTGTTC-1-5    FAS+SERPINA1+USP22+TNFRSF11A+MALT1+CD70+CD40LG...
TTTGTTGTCCTTCTTC-1-5                          IKBKB+B2M+WAS+non-targeting
TTTGTTGTCTATCGTT-1-5    TIGIT+CCR5+VEGFA+VPS13B+CD4+ATP7B+EGR3+SERPINA...
TTTGTTGTCTTAAGGC-1-5                                  BCL10+PPP1R12C+JAG1
Name: perturbation_name, Length: 204374, dtype: object

## ⚠ not sure about the `crispr_type`.

In [ ]:
adata.obs['organism'] = ORGANISMS[0]
adata.obs['crispr_type'] = 'CRISPR KO'
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = 'T cell'

In [ ]:
adata.obs['batch'].value_counts()

batch
2    36112
5    35926
0    35107
1    35074
3    33950
4    28205
Name: count, dtype: int64

In [ ]:
adata.obs['condition'] = adata.obs['perturbation_name'].apply(lambda x: 'non-targeting' if x == 'non-targeting' else 'targeting')
adata.obs['condition'].value_counts()

condition
targeting        204105
non-targeting       269
Name: count, dtype: int64

In [ ]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [ ]:
adata.write_h5ad("./GSE241683_GSE241882_cropseq.h5ad")

# SubSeries GSE241839 Pilot-data

In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE241839&format=file" -O GSE241839_RAW.tar

--2024-07-23 10:50:19--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE241839&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2851420160 (2.7G) [application/x-tar]
Saving to: ‘GSE241839_RAW.tar’

GSE241839_RAW.tar   100%[===================>]   2.66G  19.5MB/s    in 3m 57s  

2024-07-23 10:54:17 (11.5 MB/s) - ‘GSE241839_RAW.tar’ saved [2851420160/2851420160]



In [ ]:
!tar -tf GSE241839_RAW.tar

GSM7744300_GUIDEvsNT_CHR14_RESULTS.txt.gz
GSM7744300_TRAC_aneuploidy_breakpoints_ALLChromo.csv.gz
GSM7744300_cas9ProcessedAneuploidyStatus.h5ad.gz
GSM7744300_inferCNVgeneName.txt.gz
GSM7744300_rawSingletForR15Kcells.h5ad.gz


In [ ]:
with tarfile.open("GSE241839_RAW.tar") as tar:
    tar.extract("GSM7744300_cas9ProcessedAneuploidyStatus.h5ad.gz")
    tar.extract("GSM7744300_rawSingletForR15Kcells.h5ad.gz")

In [ ]:
!gzip -d 'GSM7744300_cas9ProcessedAneuploidyStatus.h5ad.gz'

In [ ]:
adata = sc.read_h5ad("GSM7744300_cas9ProcessedAneuploidyStatus.h5ad")
adata

AnnData object with n_obs × n_vars = 24483 × 23213
    obs: 'well', 'batch', 'Barcode', 'demux_type', 'assignment', 'guides', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'chr14_residual_expression', 'chr14SingleChrLoss', 'chr14DoubleChrLoss', 'chr14Status', 'chr14Aneuploidy'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: 'guides_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap', 'well_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata.obs.head()

,well,batch,Barcode,demux_type,assignment,guides,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,leiden,chr14_residual_expression,chr14SingleChrLoss,chr14DoubleChrLoss,chr14Status,chr14Aneuploidy
AAACCCACAAGTCATC-1-0,gem_well1,0,AAACCCACAAGTCATC-1-0,singlet,sample_3,GACACCTTCTTCCCCAGCCC_+,5465,5464,32739.0,1223.0,3.735606,15,1.017757,Normal,Normal,Normal,Normal
AAACCCACACACACTA-1-0,gem_well1,0,AAACCCACACACACTA-1-0,singlet,sample_4,TTCTTCCCCAGCCCAGGTAA_+,1755,1755,5627.0,433.0,7.695042,1,0.957014,Normal,Normal,Normal,Normal
AAACCCACACCCATAA-1-0,gem_well1,0,AAACCCACACCCATAA-1-0,singlet,sample_3,GACACCTTCTTCCCCAGCCC_+,5677,5676,32350.0,1356.0,4.191654,13,0.868897,Normal,chr14 loss of 2 copies,chr14 loss of 2 copies,Loss
AAACCCACACGCACCA-1-0,gem_well1,0,AAACCCACACGCACCA-1-0,singlet,sample_3,GACACCTTCTTCCCCAGCCC_+,978,978,1865.0,165.0,8.847185,14,0.979507,Normal,Normal,Normal,Normal
AAACCCACACTGCGAC-1-0,gem_well1,0,AAACCCACACTGCGAC-1-0,singlet,sample_1,AACAAATGTGTCACAAAGTA_+,2512,2512,11139.0,337.0,3.025406,3,1.003965,Normal,Normal,Normal,Normal


In [ ]:
with tarfile.open("GSE241839_RAW.tar") as tar:
    tar.extract("GSM7744300_GUIDEvsNT_CHR14_RESULTS.txt.gz")

!gzip -d GSM7744300_GUIDEvsNT_CHR14_RESULTS.txt.gz

guide_nt_results = pd.read_csv("GSM7744300_GUIDEvsNT_CHR14_RESULTS.txt", sep='\t')
guide_nt_results.head()

,AAACCCAAGCTAATGA-1-0,AAACCCAAGTATGTAG-1-0,AAACCCACAAGTCATC-1-0,AAACCCACACACACTA-1-0,AAACCCACACCCATAA-1-0,AAACCCACACGCACCA-1-0,AAACCCACACTGCGAC-1-0,AAACCCACAGGACTTT-1-0,AAACCCAGTTGGGTTT-1-0,AAACCCATCAGGTAAA-1-0,...,TTTGTTGCAATCCTTT-1-2,TTTGTTGCATGACTAC-1-2,TTTGTTGGTCAAGTTC-1-2,TTTGTTGGTGGATTTC-1-2,TTTGTTGGTTACAGCT-1-2,TTTGTTGGTTCTTGTT-1-2,TTTGTTGGTTGCCGAC-1-2,TTTGTTGTCCGGTAGC-1-2,TTTGTTGTCTAGAACC-1-2,TTTGTTGTCTTCTGGC-1-2
NOC2L,1.0012,0.849651,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,...,1.0012,1.0012,1.0012,1.0012,0.899585,0.858823,1.0012,1.0012,1.0012,1.0012
ISG15,1.0012,0.850954,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,...,1.0012,1.0012,1.0012,1.0012,0.899605,0.859515,1.0012,1.0012,1.0012,1.0012
TNFRSF4,1.0012,0.852281,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,...,1.0012,1.0012,1.0012,1.0012,0.899783,0.860290,1.0012,1.0012,1.0012,1.0012
SDF4,1.0012,0.853454,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,...,1.0012,1.0012,1.0012,1.0012,0.899916,0.860964,1.0012,1.0012,1.0012,1.0012
B3GALT6,1.0012,0.854663,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,...,1.0012,1.0012,1.0012,1.0012,0.900189,0.861720,1.0012,1.0012,1.0012,1.0012


In [ ]:
guide_nt_results.T.head()

,NOC2L,ISG15,TNFRSF4,SDF4,B3GALT6,UBE2J2,ACAP3,PUSL1,INTS11,AURKAIP1,...,PIM3,TRABD,DENND6B,PPP6R2,SBF1,LMF2,NCAPH2,SCO2,TYMP,CHKB
AAACCCAAGCTAATGA-1-0,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,...,1.0012,1.0012,1.0012,1.0012,1.0012,1.001200,1.001200,1.001200,1.001200,1.001200
AAACCCAAGTATGTAG-1-0,0.849651,0.850954,0.852281,0.853454,0.854663,0.855738,0.856733,0.857581,0.859033,0.860443,...,1.0012,1.0012,1.0012,1.0012,1.0012,1.073241,1.076885,1.080986,1.085581,1.090276
AAACCCACAAGTCATC-1-0,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,...,1.0012,1.0012,1.0012,1.0012,1.0012,1.001200,1.001200,1.001200,1.001200,1.001200
AAACCCACACACACTA-1-0,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,...,1.0012,1.0012,1.0012,1.0012,1.0012,1.001200,1.001200,1.001200,1.001200,1.001200
AAACCCACACCCATAA-1-0,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,1.001200,...,1.0012,1.0012,1.0012,1.0012,1.0012,1.001200,1.001200,1.001200,1.001200,1.001200


In [ ]:
!tar -xvf GSE241839_RAW.tar GSM7744300_TRAC_aneuploidy_breakpoints_ALLChromo.csv.gz
!gzip -d GSM7744300_TRAC_aneuploidy_breakpoints_ALLChromo.csv.gz

GSM7744300_TRAC_aneuploidy_breakpoints_ALLChromo.csv.gz


In [ ]:
!head GSM7744300_TRAC_aneuploidy_breakpoints_ALLChromo.csv

,well,batch,Barcode,demux_type,assignment,guides,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,leiden,chr14_residual_expression,chr14SingleChrLoss,chr14DoubleChrLoss,chr14Status,chr14Aneuploidy,target,target_chrom,has_infercnv,lost_chroms,chr1_bp_loc,chr1_bp_n,chr1_left,chr1_right,chr1_left_binary,chr1_right_binary,chr1_left_n,chr1_right_n,chr1_binary,chr10_bp_loc,chr10_bp_n,chr10_left,chr10_right,chr10_left_binary,chr10_right_binary,chr10_left_n,chr10_right_n,chr10_binary,chr11_bp_loc,chr11_bp_n,chr11_left,chr11_right,chr11_left_binary,chr11_right_binary,chr11_left_n,chr11_right_n,chr11_binary,chr12_bp_loc,chr12_bp_n,chr12_left,chr12_right,chr12_left_binary,chr12_right_binary,chr12_left_n,chr12_right_n,chr12_binary,chr13_bp_loc,chr13_bp_n,chr13_left,chr13_right,chr13_left_binary,chr13_right_binary,chr13_left_n,chr13_right_n,chr13_binary,chr14_bp_loc,chr14_bp_n,chr14_left,chr14_right,chr14_left_binary,chr14_right_binary,chr14_left_n,chr14_right_n,chr14_binary,chr

In [ ]:
trac_grna = pd.read_excel('./mmc1.xlsx', sheet_name='Table 1. TRAC-targeting gRNAs')
trac_grna

,gRNA,Target,Spacer
0,1,TRAC,AACAAATGTGTCACAAAGTA
1,2,TRAC,TGTGCTAGACATGAGGTCTA
2,3,TRAC,CTTCAAGAGCAACAGTGCTG
3,4,TRAC,GACACCTTCTTCCCCAGCCC
4,5,TRAC,TTCTTCCCCAGCCCAGGTAA
5,6,TRAC,CTTCTTCCCCAGCCCAGGTA
6,7,TRAC,GCTGGTACACGGCAGGGTCA
7,8,TRAC,GAGAATCAAAATCGGTGAAT
8,9,TRAC,TCTCTCAGCTGGTACACGGC
9,10,TRAC,CTTACCTGGGCTGGGGAAGA


In [ ]:
adata.obs['guides'].value_counts()

guides
CTTCAAGAGCAACAGTGCTG_+               2757
CTCTCAGCTGGTACACGGCA_-               2481
TCTCTCAGCTGGTACACGGC_-               2210
TTCTTCCCCAGCCCAGGTAA_+               2141
CTTACCTGGGCTGGGGAAGA_-               2114
GACACCTTCTTCCCCAGCCC_+               2074
GCTGGTACACGGCAGGGTCA_-               2055
AACAAATGTGTCACAAAGTA_+               2054
TGTGCTAGACATGAGGTCTA_BioRxiv/June    1946
GAGAATCAAAATCGGTGAAT_-               1782
Non-targeting                        1714
CTTCTTCCCCAGCCCAGGTA_+               1155
Name: count, dtype: int64

In [55]:
spacer_to_grna = dict(zip(trac_grna['Spacer'], trac_grna['gRNA'].astype(str)))
spacer_to_grna['Non-targeting'] = 'NT'
adata.obs['guide'] = adata.obs['guides'].apply(lambda x: spacer_to_grna.get(x.split('_')[0]))
adata.obs['guide'] = adata.obs['guide'].apply(lambda x: f"TRAC-{x}" if x != 'NT' else 'non-targeting')
adata.obs['guide'] = pd.Categorical(adata.obs['guide'], categories=[f"TRAC-{i}" for i in range(1, 12)] + ['non-targeting'])
adata.obs['guide']

AAACCCACAAGTCATC-1-0           TRAC-4
AAACCCACACACACTA-1-0           TRAC-5
AAACCCACACCCATAA-1-0           TRAC-4
AAACCCACACGCACCA-1-0           TRAC-4
AAACCCACACTGCGAC-1-0           TRAC-1
                            ...      
TTTGTTGGTGGATTTC-1-2    non-targeting
TTTGTTGGTTGCCGAC-1-2          TRAC-10
TTTGTTGTCCGGTAGC-1-2          TRAC-10
TTTGTTGTCTAGAACC-1-2           TRAC-2
TTTGTTGTCTTCTGGC-1-2          TRAC-11
Name: guide, Length: 24483, dtype: category
Categories (12, object): ['TRAC-1', 'TRAC-2', 'TRAC-3', 'TRAC-4', ..., 'TRAC-9', 'TRAC-10', 'TRAC-11',
                          'non-targeting']

In [56]:
adata.obs['perturbation_name'] = adata.obs['guide'].apply(lambda x: 'TRAC' if x != 'non-targeting' else 'non-targeting')
adata.obs['organism'] = ORGANISMS[0]
adata.obs['crispr_type'] = 'CRISPR KO'
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = 'T cell'

In [57]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 ['condition']


## chr14Aneuploidy is also a potential condition column

In [58]:
adata.obs['condition'] = adata.obs['perturbation_name'].apply(lambda x: 'non-targeting' if x == 'non-targeting' else 'targeting')
adata.write_h5ad("./GSE241683_GSE241839_pilot.h5ad")